In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 


In [70]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

In [39]:
house_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
house_data.head()
house_data.columns
#house_data = house_data.dropna(axis=1)
house_data.describe()
#house_data.isnull().sum()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [40]:
#house_data.columns

In [41]:
columns_to_fill_none = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [42]:
house_data[columns_to_fill_none] = house_data[columns_to_fill_none].fillna("None")

In [43]:
house_data[columns_to_fill_none].isnull().sum()

Alley           0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

In [44]:
ranking_Ex_Po = {
    'Ex' : 5,
    'Gd' : 4,
    'TA' : 3,
    'Fa' : 2,
    'Po' : 1,
    'None' : 0
}

In [45]:
rank_Ex_Po = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC',]

In [46]:
for col in rank_Ex_Po:
    house_data[col] = house_data[col].map(ranking_Ex_Po)

print(house_data['BsmtQual'].unique())

[4 3 5 0 2]


In [47]:
rank_GLQ_Unf = ['BsmtFinType1','BsmtFinType2',]

In [48]:
ranking_GLQ_Unf = {
    'GLQ': 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ': 2,
    'Unf': 1,
    'NA' : 0,
    'None': 0
}

In [49]:
for col in rank_GLQ_Unf:
    house_data[col] = house_data[col].map(ranking_GLQ_Unf)
    house_data.loc[house_data[col].isnull(),col] = 0


print(house_data['BsmtFinType1'].unique())

[6 5 1 3 4 0 2]


In [50]:
#maping for Alley
house_data['Alley'] = house_data['Alley'].map({'Grvl': 2, 'Pave': 1, 'None': 0})
print(house_data['Alley'].unique())

[0 2 1]


In [51]:
#maping for streets
house_data['Street'] = house_data['Street'].map({'Grvl': 0,'Pave': 1})
print(house_data['Street'].unique())

[1 0]


In [52]:
#maping lot shape
house_data['LotShape'] = house_data['LotShape'].map({'IR3': 0,'IR2':1,'IR1':2,'Reg':3})

In [53]:
#maping LandContour
house_data['LandContour'] = house_data['LandContour'].map({'low':0,'HLS':1,'Bnk':2,'Lvl':3})

In [54]:
#maping Utilities
house_data['Utilities'] = house_data['Utilities'].map({'ELO':0,'NoSeWa':1,'NoSewr':2,'AllPub':3})

In [55]:
#maping 
house_data['LandSlope'] = house_data['LandSlope'].map({'Gtl':2,'Mod':1,'Sev':0})

In [56]:
#maping 
house_data['Electrical'] = house_data['Electrical'].map({'Mix':0,'FuseP':1,'Fuse':2,'FuseA':3,'SBrkr':4})

In [57]:
#maping 
house_data['Functional'] = house_data['Functional'].map({'Sal':0,'Sev':1,'Maj2':2,'Maj1':3,'Mod':4,'Min2':5,'Min1':6,'Typ':7})

In [58]:
#maping 
house_data['GarageFinish'] = house_data['GarageFinish'].map({'NA':0,'Unf':1,'RFn':2,'Fin':3})

In [59]:
#maping 
house_data['PavedDrive'] = house_data['PavedDrive'].map({'Y':2,'N':0,'P':1})

In [60]:
house_data['Fence'] = house_data['Fence'].map({'NA':0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})

In [61]:
nominal_cols = house_data.select_dtypes(include=['object']).columns
print(nominal_cols)

Index(['MSZoning', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure', 'Heating',
       'CentralAir', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition'],
      dtype='object')


In [62]:
house_data = pd.get_dummies(house_data, columns=nominal_cols, drop_first=True)
#print(house_data.shape)

In [63]:
#pd.set_option('display.max_columns', None)
#print(house_data.tail())

In [66]:
X = house_data.drop('SalePrice', axis=1)

y = house_data['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X , y ,test_size=0.2, random_state=1)

pipeline = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    LinearRegression()
)

pipeline.fit(X_train,y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [68]:
house_prediction = pipeline.predict(X_test)
print(house_prediction[:10])

[216837.02949378 164321.86231861 111169.11070234  74164.30329177
 125464.58692846 352664.04853964 317867.85577397 138887.23420721
 202481.26601583 266991.09538116]


In [73]:
mse = mean_squared_error(y_test,house_prediction)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test,house_prediction)
r2 = r2_score(y_test, house_prediction)

print("RSME", rmse)
print("MAE", mae)
print("R2", r2)

RSME 2.127061008245823e+16
MAE 1244782312475462.0
R2 -6.343836867370484e+22
